In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
b=pd.read_csv('/content/drive/MyDrive/twitter_training.csv')
b

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [ ]:
b.columns=['id','game','sentiment','tweet']
b.head()

,id,game,sentiment,tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
b.tail()

,id,game,sentiment,tweet
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...
74680,9200,Nvidia,Positive,Just like the windows partition of my Mac is l...


In [ ]:
b = b.dropna(subset=['tweet'])

In [ ]:
b['sentiment'].value_counts()

Negative      22358
Positive      20654
Neutral       18108
Irrelevant    12875
Name: sentiment, dtype: int64

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

def process_tweet(tweet):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
                word not in string.punctuation):
            stem_word = stemmer.stem(word)
            tweets_clean.append(stem_word)

    return tweets_clean

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
process_tweet(b['tweet'][1])

['im', 'get', 'borderland', 'kill']

In [ ]:
def count_tweets(result, tweets, ys):

    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)

            if pair in result:
                result[pair] += 1

            else:
                result[pair] = 1

    return result

In [ ]:
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am not tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

In [ ]:
b.loc[:, 'prep'] = b['tweet'].apply(process_tweet)

<ipython-input-15-2d047ffe6030>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b.loc[:, 'prep'] = b['tweet'].apply(process_tweet)


In [ ]:
sentiment_mapping = {
    'Positive': 1,
    'Negative': 2,
    'Neutral': 3,
    'Irrelevant': 4
}

b.loc[:, 'label'] = b['sentiment'].map(sentiment_mapping)
b.head()

<ipython-input-16-3ec07a5cad1b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b.loc[:, 'label'] = b['sentiment'].map(sentiment_mapping)


,id,game,sentiment,tweet,prep,label
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,"[come, border, kill]",1
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,"[im, get, borderland, kill]",1
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,"[im, come, borderland, murder]",1
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,"[im, get, borderland, 2, murder]",1
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...,"[im, get, borderland, murder]",1


In [ ]:
freqs = count_tweets({}, b['tweet'], b['label'])

In [ ]:
def train_naive_bayes(freqs, train_x, train_y):

    loglikelihoods = {}
    logpriors = {}

    vocab = set([key[0] for key in freqs.keys()])
    V = len(vocab)

    for label in set(train_y):
        N_label = sum(freqs.get((word, label), 0) for word in vocab)
        V_label = len([word for word in vocab if freqs.get((word, label), 0) > 0])

        D_label = sum(1 for l in train_y if l == label)

        logpriors[label] = np.log(D_label / len(train_y))

        for word in vocab:
            freq_label = freqs.get((word, label), 0)

            p_w_label = (freq_label + 1) / (N_label + V)

            if word not in loglikelihoods:
                loglikelihoods[word] = {}
            loglikelihoods[word][label] = np.log(p_w_label)

    return logpriors, loglikelihoods

In [ ]:
logprior, loglikelihood = train_naive_bayes(freqs, b['tweet'], b['label'])
print(logprior)
print(len(loglikelihood))

{1: -1.276088518104599, 2: -1.1968133245056773, 3: -1.4076436937800991, 4: -1.7487100768040114}
32942


In [ ]:
def naive_bayes_predict(tweet, logprior, loglikelihood):

    processed_tweet = process_tweet(tweet)

    probability = logprior.copy()

    for word in processed_tweet:
        if word in loglikelihood and loglikelihood[word]:
            for label, log_likelihood in loglikelihood[word].items():
                probability[label] += log_likelihood

    predicted_label = max(probability, key=probability.get)

    return predicted_label

In [ ]:
my_tweet = "i have played this for over 50 hours, i love this game"
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1


In [ ]:
test_b=pd.read_csv('/content/drive/MyDrive/twitter_validation.csv')
test_b.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [ ]:
test_b.columns=['id','game','sentiment','tweet']
sentiment_mapping = {
    'Positive': 1,
    'Negative': 2,
    'Neutral': 3,
    'Irrelevant': 4
}

test_b.loc[:, 'label'] = test_b['sentiment'].map(sentiment_mapping)
test_b.head()

,id,game,sentiment,tweet,label
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...,3
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...,2
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,...",2
3,4433,Google,Neutral,Now the President is slapping Americans in the...,3
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...,2


In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):

    correct_predictions = 0

    for tweet, true_label in zip(test_x, test_y):
        predicted_label = naive_bayes_predict(tweet, logprior, loglikelihood)

        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_y)

    return accuracy

test_accuracy = test_naive_bayes(test_b['tweet'], test_b['label'], logprior, loglikelihood)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.7827827827827828


In [ ]:
my_tweet = input("enter the text")
naive_bayes_predict(my_tweet, logprior, loglikelihood)

enter the textyou are so bad at this :(


2